In [1]:
import tensorflow as tf
import numpy 
import pandas 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
seed = 7
numpy.random.seed(seed)

In [3]:
# load dataset
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]



In [4]:
encoder=LabelEncoder()
encoder.fit(Y)
encoded_Y=encoder.transform(Y)
mine=1
rock=0

In [5]:
print(dataframe)
print(X)
print("********************************")
print(Y)

         0       1       2       3       4       5       6       7       8   \
0    0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1    0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2    0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3    0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4    0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   
5    0.0286  0.0453  0.0277  0.0174  0.0384  0.0990  0.1201  0.1833  0.2105   
6    0.0317  0.0956  0.1321  0.1408  0.1674  0.1710  0.0731  0.1401  0.2083   
7    0.0519  0.0548  0.0842  0.0319  0.1158  0.0922  0.1027  0.0613  0.1465   
8    0.0223  0.0375  0.0484  0.0475  0.0647  0.0591  0.0753  0.0098  0.0684   
9    0.0164  0.0173  0.0347  0.0070  0.0187  0.0671  0.1056  0.0697  0.0962   
10   0.0039  0.0063  0.0152  0.0336  0.0310  0.0284  0.0396  0.0272  0.0323   
11   0.0123  0.0309  0.0169  0.0313  0.0358  0.0102 

**Baseline Neural Network Model Performance**

In [6]:
# baseline model
def create_baseline():
  # create model, write code below
    from keras import models
    from keras import layers
    model=tf.keras.Sequential()
  #input layer
    model.add(Dense(60,activation='relu',input_shape=(60,)))
  #output layer
    model.add(Dense(1,activation='sigmoid'))
	# Compile model, write code below
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model


In [7]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X,encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Using TensorFlow backend.


Results: 83.68% (4.89%)


**Re-Run The Baseline Model With Data Preparation**

In [19]:
# baseline model
def create_baseline():
  # create model, write code below
    from keras import models
    from keras import layers
    model=tf.keras.Sequential()
  #input layer
    model.add(Dense(60,activation='relu',input_shape=(60,)))
  #output layer
    model.add(Dense(1,activation='sigmoid'))
	# Compile model, write code below
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model


In [20]:

# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X,encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 85.52% (6.95%)


# Tuning Layers and Number of Neurons in The Model

**Evaluate a Smaller Network**

In [11]:
# smaller model
def create_smaller():
  # create model
    from keras import models
    from keras import layers
    model=tf.keras.Sequential()
  #input layer
    model.add(Dense(30,activation='relu',input_shape=(60,)))
  #output layer
    model.add(Dense(1,activation='sigmoid'))
	
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
	# Compile model
    return model


In [12]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 85.54% (6.19%)


[0.86363637 0.95238096 0.76190478 0.85714287 0.90476191 0.80952382
 0.95238096 0.75       0.80000001 0.89999998]


**Evaluate a Larger Network**

In [13]:
# larger model
def create_larger():
   # create model
    from keras import models
    from keras import layers
    model=tf.keras.Sequential()
  #input layer
    model.add(Dense(60,activation='relu',input_shape=(60,)))
    model.add(Dense(30,activation='relu'))
  #output layer
    model.add(Dense(1,activation='sigmoid'))
	
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
	# Compile model
  
    return model


In [14]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 84.63% (7.04%)


**Really Scaling up: developing a model that overfits**

In [15]:
# larger model
def create_larger():
   # create model
    from keras import models
    from keras import layers
    model=tf.keras.Sequential()
  #input layer
    model.add(Dense(80,activation='relu',input_shape=(60,)))
    model.add(Dense(60,activation='relu'))
    model.add(Dense(30,activation='relu'))
  #output layer
    model.add(Dense(1,activation='sigmoid'))
	
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
	# Compile model
  
    return model

In [16]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=800, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 84.61% (7.35%)


**Tuning the Model**

In [17]:
# larger model
def create_Tunning_model():
   # create model
    from keras import models
    from keras import layers
    model=tf.keras.Sequential()
  #input layer
    model.add(Dense(80,activation='relu',input_shape=(60,)))
    model.add(Dense(60,activation='relu'))
    model.add(Dense(30,activation='relu'))

            #output layer
    model.add(Dense(1,activation='sigmoid'))
	
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
	# Compile model
    return model

In [18]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_Tunning_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Tunnig: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Tunnig: 84.59% (7.13%)


**Rewriting the code using the Keras Functional API**

In [28]:
#import keras 
#from keras import models
#from keras import layers
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
inputs=Input(shape=(60,))
x=Dense(60,activation='relu')(inputs)
outputs=Dense(1,activation='sigmoid')(x)
model=Model(inputs=inputs,outputs=outputs)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history=model.fit(X, encoded_Y,epochs=100, batch_size=5, verbose=0)
#print("Tunnig: %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))
history_dict=history.history
history_dict.keys()
acc_values=history_dict['accuracy']

print("Result: %.2f%% (%.2f%%)" % (numpy.mean(acc_values)*100, numpy.std(acc_values)*100))


Result: 88.62% (7.20%)


**Rewriting the code by doing Model Subclassing**

In [32]:

import keras
from keras import layers
from keras import models
class My_model(tf.keras.Model):
    def __init__(self):
        super(My_model,self).__init__()
        self.dense1=Dense(60,activation='relu')
        self.dense2=Dense(30,activation='relu')
        self.dense3=Dense(1,activation='sigmoid')
    def call(self,x):
        x=self.dense1(x)
        x=self.dense2(x)
        return self.dense3(x)

model=My_model()
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
history=model.fit(X,encoded_Y, epochs=100,batch_size=5,verbose=False)

history_dict=history.history
history_dict.keys()
acc_values=history_dict['accuracy']
print("Result: %.2f%% (%.2f%%)" % (numpy.mean(acc_values)*100, numpy.std(acc_values)*100))



Result: 93.69% (8.28%)


**Rewriting the code without using scikit-learn**

In [47]:
def create_model():
    from keras import models
    from keras import layers
    model=tf.keras.Sequential()
    model.add(Dense(60,activation='relu',input_shape=(60,)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  
    return model

In [54]:
k=10
num_val_samples =len(X)//k
all_score=[]
for i in range(k):
    print('processing fold #', i)
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
#     print(val_data,val_targets)
    train_data =numpy.concatenate([X[:i * num_val_samples],X[(i + 1) * num_val_samples:]],axis=0)
    train_targets =numpy.concatenate([encoded_Y[:i * num_val_samples],encoded_Y[(i + 1) * num_val_samples:]],axis=0)
    model = create_model()
    model.fit(train_data, train_targets,epochs=50, batch_size=50,verbose=0)
    loss,acc=model.evaluate(val_data,val_targets,verbose=0)
#     val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
#     all_score.append(val_mae)
# print("Result: %.2f%% (%.2f%%)" % (numpy.mean(all_score)*100, numpy.std(all_score)*100))

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [56]:
print(loss)
print(acc)
model.save("simple_keras_model.h5")

0.3440415859222412
0.85
